# Question 1.
- 학생성적 예측 데이터
- 데이터를 병합하지 않고 풀이

In [11]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv')

# 결과변수 설정
y_train = y_train['G3']

# 결측치 확인
display(x_train.isna().sum())
display(x_test.isna().sum())

# 통계량 확인
display(x_train.describe(exclude = 'number'))
display(x_train.describe())

StudentID     0
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
dtype: int64

StudentID     0
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
dtype: int64

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,678,678,678,678,678,678,678,678,678,678,678,678,678,678,678,678,678
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,no,yes,yes,yes,no
freq,509,394,499,479,602,254,371,269,475,604,419,537,350,543,621,541,433


,StudentID,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
count,678.000000,678.000000,678.000000,678.000000,678.00000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.000000,678.00000,678.000000
mean,1516.181416,16.719764,2.607670,2.371681,1.50590,1.989676,0.261062,3.933628,3.233038,3.140118,1.480826,2.293510,3.522124,4.594395,11.20649,11.182891
std,303.029891,1.239881,1.134682,1.096750,0.73675,0.848082,0.664340,0.952346,1.042083,1.165537,0.895789,1.287705,1.431430,6.527236,2.90395,3.296146
min,1001.000000,15.000000,0.000000,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,4.00000,0.000000
25%,1253.250000,16.000000,2.000000,1.000000,1.00000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,2.000000,0.000000,9.00000,9.000000
50%,1508.000000,17.000000,3.000000,2.000000,1.00000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,2.000000,11.00000,11.000000
75%,1780.750000,18.000000,4.000000,3.000000,2.00000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,6.000000,13.00000,13.000000
max,2043.000000,22.000000,4.000000,4.000000,4.00000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.00000,19.000000


## 데이터를 수치형과 범주형으로 나누어 전처리

In [12]:
train_cat = x_train.select_dtypes(exclude = 'number')
train_num = x_train.select_dtypes(include = 'number')
test_cat = x_test.select_dtypes(exclude = 'number')
test_num = x_test.select_dtypes(include = 'number')

## 결측치 처리

In [21]:
train_num = train_num.apply(lambda x: x.fillna(x.median(), inplace = False))
train_cat = train_cat.apply(lambda x: x.fillna(x.mode()[0], inplace = False))
test_num = test_num.apply(lambda x: x.fillna(x.median(), inplace = False))
test_cat = test_cat.apply(lambda x: x.fillna(x.mode()[0], inplace = False))

## Encoding

In [24]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False
)

train_encoded = ohe.fit_transform(train_cat)
train_cat = pd.DataFrame(
    train_encoded,
    columns = ohe.get_feature_names_out(train_cat.columns.tolist()),
    index = train_cat.index
)

test_encoded = ohe.transform(test_cat)
test_cat = pd.DataFrame(
    test_encoded,
    columns = ohe.get_feature_names_out(test_cat.columns.tolist()),
    index = test_cat.index
)

X = pd.concat([train_num, train_cat], axis = 1)
test_final = pd.concat([test_num, test_cat], axis = 1)

## 데이터 분할

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size = 0.3, random_state = 42)

## 모델 선언 및 학습

In [27]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 모델 예측 및 평가지표 출력

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

val_pred = rf.predict(X_val)

# 평가 지표 출력
print("회귀 모델 평가 지표")
print(f"MAE   : {mean_absolute_error(y_val, val_pred):.4f}")
print(f"MSE   : {mean_squared_error(y_val, val_pred):.4f}")
print(f"RMSE  : {np.sqrt(mean_squared_error(y_val, val_pred)):.4f}")
print(f"R^2   : {r2_score(y_val, val_pred):.4f}")

회귀 모델 평가 지표
MAE   : 1.0114
MSE   : 2.4539
RMSE  : 1.5665
R^2   : 0.8270


## CSV 파일 생성

In [32]:
test_pred = rf.predict(test_final)

pd.DataFrame({'pred' : test_pred}).to_csv('ans_csv', index = True, index_label = 'ID')